<h1> 1. Parameters, Imports, and Inputs

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
histogram_bins = 1000
samples = 1000000
training_samples = int(0.9*samples)
testing_samples = int(0.1*samples)

In [2]:
mean = 0 #input("Please input the mean of the gaussian curve:")
std_dev = 1 #input("Please input the stardard deviation of the gaussian curve:")

<h1> 2. Dataset and Histogramming </h1>

In [3]:
raw_dataset = np.random.normal(mean, std_dev,samples)
starting_points = np.linspace(np.min(raw_dataset), np.max(raw_dataset), num=histogram_bins, endpoint=False)
mid_points = np.zeros(histogram_bins)
count = np.zeros(histogram_bins)

start_time = time.time()
for i in range(histogram_bins):
    #Edge cases
    if i == histogram_bins-1:
        maximum = np.max(raw_dataset)
    else:
        maximum = starting_points[i+1]
    
    #Get the midpoints
    mid_points[i] = starting_points[i]+(starting_points[i] - maximum)/2
    
    #Sweep through the dataset
    for j in raw_dataset:
        if ((j >= starting_points[i]) and (j <= maximum)): #Assume no EXACTLY equal number
            count[i] = count[i] + 1
    
end_time = time.time() - start_time
print("Time took: "+str(end_time))
print("Total number of samples check:"+str(np.sum(count)))

Time took: 241.71027326583862
Total number of samples check:1000000.0


In [4]:
def translate_count(count, mid_points):
    output = []
    for index, occurrence in enumerate(count):
        if(occurrence !=0):
            for _ in range(int(occurrence)):
                output.append(mid_points[index])
    output_array = np.array(output)
    return output_array

dataset = translate_count(count,mid_points)
random_dataset = np.random.choice(dataset,samples,replace=False)
training_dataset = random_dataset[0:training_samples]
testing_dataset = random_dataset[training_samples:(training_samples + testing_samples)]

print("Size of testing dataset (histogrammed):"+str(training_dataset.shape))
print("Size of testing dataset (histogrammed):"+str(testing_dataset.shape))

Size of testing dataset (histogrammed):(900000,)
Size of testing dataset (histogrammed):(100000,)


<h1> 3. Build a 3 layer neural network 

In [41]:
nn_architecture = [
    {"input_dim": 1, "output_dim": 64, "activation": "relu"},
    {"input_dim": 64, "output_dim": 64, "activation": "relu"},
    {"input_dim": 64, "output_dim": 64, "activation": "relu"},
    {"input_dim": 64, "output_dim": 1, "activation": "relu"},
]

def init_layers(nn_architecture, seed = 99):
    np.random.seed(seed)
    number_of_layers = len(nn_architecture)
    params_values = {}

    for idx, layer in enumerate(nn_architecture):
        layer_idx = idx + 1
        layer_input_size = layer["input_dim"]
        layer_output_size = layer["output_dim"]
        
        params_values['W' + str(layer_idx)] = np.random.normal(0,0.1,(layer_output_size, layer_input_size))
        params_values['b' + str(layer_idx)] = np.random.normal(0,0.1,(layer_output_size, 1))
        
    return params_values

def sigmoid(Z):
    return 1/(1+np.exp(-Z))

def relu(Z):
    return np.maximum(0,Z)

def sigmoid_backward(dA, Z):
    sig = sigmoid(Z)
    return dA * sig * (1 - sig)

def relu_backward(dA, Z):
    dZ = np.array(dA, copy = True)
    dZ[Z <= 0] = 0;
    return dZ;

def single_layer_forward_propagation(A_prev, W_curr, b_curr, activation="relu"):
    Z_curr = np.dot(W_curr, A_prev) + b_curr
    
    if activation == "relu":
        activation_func = relu
    elif activation == "sigmoid":
        activation_func = sigmoid
    else:
        raise Exception('Non-supported activation function')
        
    return activation_func(Z_curr), Z_curr

def full_forward_propagation(X, params_values, nn_architecture):
    memory = {}
    A_curr = X
    
    for idx, layer in enumerate(nn_architecture):
        layer_idx = idx + 1
        A_prev = A_curr
        
        activ_function_curr = layer["activation"]
        W_curr = params_values["W" + str(layer_idx)]
        
        b_curr = params_values["b" + str(layer_idx)]
        A_curr, Z_curr = single_layer_forward_propagation(A_prev, W_curr, b_curr, activ_function_curr)
        
        memory["A" + str(idx)] = A_prev
        memory["Z" + str(layer_idx)] = Z_curr
       
    return A_curr, memory

def get_cost_value(predictions,targets):
    # Retrieving number of samples in dataset
    samples_num = len(predictions)
    
    # Summing square differences between predicted and expected values
    accumulated_error = 0.0
    for prediction, target in zip(predictions, targets):
        accumulated_error += (prediction - target)**2
        
    # Calculating mean and dividing by 2
    mae_error = (1.0 / (2*samples_num)) * accumulated_error
    
    return mae_error

def get_accuracy_value(Y_hat, Y):
    return 100 - np.mean(np.abs(Y_hat - Y)) * 100

def single_layer_backward_propagation(dA_curr, W_curr, b_curr, Z_curr, A_prev, activation="relu"):
    m = A_prev.shape[1]
    
    if activation == "relu":
        backward_activation_func = relu_backward
    elif activation == "sigmoid":
        backward_activation_func = sigmoid_backward
    else:
        raise Exception('Non-supported activation function')
    
    dZ_curr = backward_activation_func(dA_curr, Z_curr)
    dW_curr = np.dot(dZ_curr, A_prev.T) / m
    db_curr = np.sum(dZ_curr, axis=1, keepdims=True) / m
    dA_prev = np.dot(W_curr.T, dZ_curr)

    return dA_prev, dW_curr, db_curr

def full_backward_propagation(Y_hat, Y, memory, params_values, nn_architecture):
    grads_values = {}
    m = Y.shape[1]
    Y = Y.reshape(Y_hat.shape)
    
    dA_prev = (Y_hat - Y)
    
    for layer_idx_prev, layer in reversed(list(enumerate(nn_architecture))):
        layer_idx_curr = layer_idx_prev + 1
        activ_function_curr = layer["activation"]
        
        dA_curr = dA_prev
        
        A_prev = memory["A" + str(layer_idx_prev)]
        Z_curr = memory["Z" + str(layer_idx_curr)]
        W_curr = params_values["W" + str(layer_idx_curr)]
        b_curr = params_values["b" + str(layer_idx_curr)]
        
        dA_prev, dW_curr, db_curr = single_layer_backward_propagation(
            dA_curr, W_curr, b_curr, Z_curr, A_prev, activ_function_curr)
        
        grads_values["dW" + str(layer_idx_curr)] = dW_curr
        grads_values["db" + str(layer_idx_curr)] = db_curr
        
    
    return grads_values

def update(params_values, grads_values, nn_architecture, learning_rate):
    for layer_idx, layer in enumerate(nn_architecture):
        params_values["W" + str(layer_idx+1)] -= learning_rate * grads_values["dW" + str(layer_idx+1)]        
        params_values["b" + str(layer_idx+1)] -= learning_rate * grads_values["db" + str(layer_idx+1)]

    return params_values;

def train(X, Y, nn_architecture, epochs, learning_rate):
    params_values = init_layers(nn_architecture, 2)
    cost_history = []
    accuracy_history = []
    
    for i in range(epochs):
        Y_hat, cashe = full_forward_propagation(X, params_values, nn_architecture)
        cost = get_cost_value(Y_hat, Y)
        cost_history.append(cost)
        accuracy = get_accuracy_value(Y_hat, Y)
        accuracy_history.append(accuracy)
        
        grads_values = full_backward_propagation(Y_hat, Y, cashe, params_values, nn_architecture)
        
        params_values = update(params_values, grads_values, nn_architecture, learning_rate)
        
def test(X, Y, nn_architecture,params_values):
    Y_hat, cashe = full_forward_propagation(X, params_values, nn_architecture)
    cost = get_cost_value(Y_hat, Y)
    accuracy = get_accuracy_value(Y_hat, Y)

    return accuracy

def normal(x,mu,sigma):
    return ( 2.*np.pi*sigma**2. )**-.5 * np.exp( -.5 * (x-mu)**2. / sigma**2. )

def real_function(inputs,mean,std_dev):
    output = []
    for i in range(inputs.shape[0]):
        gaussian_eq = normal(inputs[i],mean,std_dev)
        output.append(relu(gaussian_eq))
    
    return np.array(output)[np.newaxis,:]


In [36]:
y = real_function(training_dataset,mean,std_dev)
params_values, cost_history, accuracy_history = train(training_dataset[np.newaxis,:],y,nn_architecture,20,0.1)
print(accuracy_history)

[75.12741103757014, 79.67488630287781, 82.83403678890062, 85.00554205898061, 86.50539734086693, 87.54935915595473, 88.27992433057585, 88.79595099401638, 89.16728983248213, 89.43949868382937, 89.64279470304155, 89.79838946834379, 89.91964767395626, 90.01711412967546, 90.0969989696036, 90.16389758548318, 90.22109342838961, 90.27178555829903, 90.31701113723148, 90.35843683431261]


In [43]:
z = real_function(testing_dataset,mean,std_dev)
print(test(testing_dataset[np.newaxis,:],z,nn_architecture,params_values))
print(test(testing_dataset[np.newaxis,:],z,nn_architecture,params_values))

90.39437670370042
90.39437670370042
